In [ ]:
import tensorflow
import mne

In [ ]:
# Import the BrainVision data into an MNE Raw object
raw = mne.io.read_raw_brainvision('path to .vhdr file')

# Read in the event information as MNE annotations
annot = mne.read_annotations('path to .vmrk file')

# Add the annotations to our raw object so we can use them with the data
raw.set_annotations(annot)

channels

raw data structure:

```
raw =  { 'channel':
            {'eeg_data': {data},
             'sth': {nvm}
            }
        }
```

In [ ]:
# Reconstruct the original events from our Raw object
events, event_ids = mne.events_from_annotations(raw)